<a href="https://colab.research.google.com/github/joycekuohmoukouri/creditworthiness-scoring/blob/main/KuohMoukouri_Joyce_D_dashboard_072023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet 7 : Implémentez un modèle de scoring

## NB D : Dashboard

- <a href="#C1"> 1. Importation des données</a>

- <a href="#C2"> 2. Préparation du dataset </a>

- <a href="#C3"> 3. Intégration du modèle </a>

- <a href="#C4"> 4. Dashboard </a>
  -  <a href="#C41"> 4.1 Dashboard layout <a>
  -  <a href="#C42"> 4.2 Visualisation <a>
  -  <a href="#C43"> 4.3 Intéractivité <a>
  -  <a href="#C44"> 4.4 Outils d'interprétation <a>


In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
import seaborn as sns
import missingno as msno
import clean #mon module de fontions qui me permettent de nettoyer un dataframe
import plot_
import joblib
import os
import MyModule_p7
import requests

#-------------------------Filter warnings-------------------------
import warnings
warnings.filterwarnings('ignore')
! pip install dash --quiet
! pip install category_encoders --quiet
! pip install shap --quiet
! pip install flask

#---------------------------- Dash ------------------------------
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import shap

In [2]:
!pip freeze > requirements.txt

# <a name="C1"> 1. Chargement du preproccesing et du modèle de classification </a>

In [3]:
from flask import Flask, request, jsonify
df_train_org = pd.read_csv('df_train_set_1.csv',
                 usecols=['SECTEUR_ACTIVITE'])

freq_by_org_type = df_train_org['SECTEUR_ACTIVITE'].value_counts(normalize=True).to_dict()

def frequency_encode(x):
    return x.replace(freq_by_org_type)


In [4]:
loaded_preprocess = MyModule_p7.preprocess_model()

In [5]:
classification_model = joblib.load('LightGBM_bestmodel.pkl')

# <a name="C2"> 2. Création des fonctions pour l'API </a>

In [15]:
def feat_local(df_client_pp):
  # J'instancie le Shap explainer -----------------------------------
  explainer = shap.Explainer(classification_model)

  # Calculate SHAP values for the client's prediction
  shap_values = explainer.shap_values(df_client_pp)
  ##-------- df_pp
  prefixes_to_remove = ['oneHot__', 'remainder__', 'frequency__']
  new_column_names = [col.replace(prefix, '') for col in df_client_pp.columns for prefix in prefixes_to_remove if col.startswith(prefix)]
  df_client_pp.columns = new_column_names
  return shap_values, df_client_pp

def prediction_credit():
    client_id = str(input("Entrer id_clients"))
    print(client_id)
    client_data = MyModule_p7.get_client_data(client_id)
    df_client_pp = loaded_preprocess.transform(client_data)
    prediction = classification_model.predict(df_client_pp)
    proba = classification_model.predict_proba(df_client_pp)
    score = int(round((1-proba[0][1])*100))
    SV, df_client_pp = feat_local(df_client_pp)
    return client_id, {'prediction': [prediction, proba]}, score, df_client_pp, SV


In [19]:
#test
prediction_credit()

Entrer id_clients100005
100005


('100005',
 {'prediction': [array(['1'], dtype=object),
   array([[0.49057614, 0.50942386]])]},
 49,
    PROPRIETAIRE_N  PROPRIETAIRE_Y  NIVEAU_ETUDE_ENS_SUP  NIVEAU_ETUDE_BAC  \
 1               0               1                     0                 1   
 
    NIVEAU_ETUDE_COLLEGE  NBRE_J_RETARD  OCCUPATION_Working  \
 1                     0       0.111111                   1   
 
    OCCUPATION_State servant  OCCUPATION_Commercial associate  \
 1                         0                                0   
 
    OCCUPATION_Pensioner  ...  AGE  ANCIENNETE_EMPLOI  SCORE_REGION  HEURE_APP  \
 1                     0  ...   50                 12             2          9   
 
    SCORE_2_EXT  RATIO_ENDETT(%)  NBRE_CONTRAT_ACTIFS  POS_PROGRESS_MAX_MIN  \
 1     0.291656             17.5                  2.0                 100.0   
 
    CC_NOMBRE_RETRAIT_MOYEN  CB_SOMME_DUES_RETARD  
 1                      0.0                   0.0  
 
 [1 rows x 31 columns],
 [array([[-0.40074433,  0

# <a name="C3"> 3. Création de l'API </a>

In [20]:
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    # Receive client ID from the request
    data = request.get_json()
    client_id = data.get('client_id')

    client_data = MyModule_p7.get_client_data(client_id)
    df_client_pp = loaded_preprocess.transform(client_data)

    # Get prediction, score, SHAP values, etc.
    result = prediction_credit(df_client_pp)

    # Return the results as a JSON response
    return jsonify({
        'client_id': result[0],
        'prediction': result[1][0],
        'proba': result[1][1],
        'score': result[2],
        'df_client_pp': result[3].to_dict(),  # Convert DataFrame to dictionary
        'shap_values': result[4].tolist()  # Convert SHAP values to list
    })


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
